# Sleep Data Analysis
Justin Yi  
4/21/25  
CPSC 222
Spring 2025
Description:  
This Jupyter Notebook explores, interprets, and learns my sleep data.

## Overview
The two datasets we'll be combining and analyzing are my personal sleep data, collected by Fitbit, and weather data courtesty of MeteoStat API. 